<a href="https://colab.research.google.com/github/James-R-Chapman/Kia-Invoices/blob/main/Kia_Invoices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install camelot-py[cv]
#!apt-get install ghostscript
#!sudo apt install tesseract-ocr
!pip install openpyxl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.1/248.1 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 56.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
import csv
import glob
import re
import camelot
import pandas as pd
from pdfminer.high_level import extract_text
from google.colab import files
import openpyxl
from google.colab import drive
import os
drive.mount('/content/drive')

# Upload the PDF file
#uploaded = files.upload()

# Get the path of the uploaded file
#pdf_path = list(uploaded.keys())[0]
#pdf_path = '/content/Batch Report (39).pdf'

pdf_folder = '/content/drive/MyDrive/Courage Kia/Kia Invoices'
csv_file_path = '/content/parts_invoices.csv'
# Check if parts_invoices.csv file already exists
if os.path.isfile(csv_file_path):
    skip = True
    print(f"{csv_file_path} already exists. Skip is set to {skip}.")
else:
    skip = False
    print(f"{csv_file_path} does not exist. Skip is set to {skip}.")

subtotal = 0
for pdf_path in glob.glob(pdf_folder + '/*.pdf'):

    if not skip:
        print(f"Reading file: {pdf_path}")
        
        # Extract the text from the PDF file using pdfminer
        text = extract_text(pdf_path)

        # Split the text into lines and remove any empty lines
        lines = [line.strip() for line in text.split('\n') if line.strip()]

        # Write the lines to the CSV file in append mode
        with open(csv_file_path, 'a', newline='') as csv_file:
            writer = csv.writer(csv_file)
            writer.writerows([line.split() for line in lines])

headers = ['Invoice Number', 'Invoice Date', 'Line #', 'Order #', 'Part Number', 'Status', 'QTY Order', 'QTY Ship', 'B/O Cancel', 'Cost', 'Ext Price']
invoice_headers = ['INVOICE #', 'DATE', 'SHIPMENT', 'CORE', 'SUBTOTAL', 'FREIGHT', 'HANDLING', 'MISC', 'RESTOCK FEE', 'TAX', 'INVOICE TOTAL']
credit_headers = ['Invoice Number', 'Invoice Date', 'Line #', 'Order #', 'Part Number', 'Type', 'QTY', 'ACT Code', 'Handling Credit', 'Cost', 'Ext Price']
kia_invoices = []
invoices = []
credits = []
prev_row = []
invoice_pages = set()  # create a set to store all unique invoice numbers encountered
invoice_no = ''
date = ''
line = ''
ro = ''
order = ''
part = ''
number = ''
part_number = ''
status = ''
qty_order = ''
act_code = ''
qty_ship = ''
handling = ''
bo_cancel = ''
cost = ''
price = ''
page = ''
core = ''
part_number = part + number
shipment = ''
invoice_core = ''
freight = ''
invoice_handling = ''
misc = ''
restock = ''
tax = ''
invoice_total = ''
prev_invoice_no = ''

for i, row in enumerate(csv.reader(open('/content/parts_invoices.csv', 'r'))):
    core = ''
    if len(row) >= 4 and (row[0].startswith('Courage') and 'CLAIM' in row[3]) or (row[0].startswith('4290') and row[2].startswith('BLVD') and row[3].startswith('I') and row[4].startswith('N')):
        invoice_no = row[5]
        invoice_core = ''
        freight = ''
        invoice_handling = ''
        misc = ''    
    elif len(row) >= 4 and row[0].startswith('DATE:') and 'PAGE:' in row[2]:
        date = row[1]
        page = row[3]
    elif len(row) >= 5 and row[0].startswith('GASTONIA') and 'DATE' in row[3]:
        date = row[4]
    elif row[0].startswith('SHIPMENT'):
        if len(row) >= 6:
            shipment = row[2]
    elif row[0].startswith('***Core'):
        if len(row) >= 4:
            invoice_core = row[3]
    elif row[0].startswith('SUB') or row[0].startswith('Sub'):
            subtotal = row[2]
    elif len(row) >= 5 and (row[3].startswith('PART') or row[3].startswith('Part')) and (row[4].startswith('TOTAL') or row[4].startswith('Total')):
            subtotal = row[5]
    elif len(row) >= 5 and (row[2].startswith('PART') or row[2].startswith('Part')) and (row[3].startswith('TOTAL') or row[3].startswith('Total')):
            subtotal = row[4]
    elif len(row) >= 4 and (row[1].startswith('PART') or row[1].startswith('Part')) and (row[2].startswith('TOTAL') or row[2].startswith('Total')):
            subtotal = row[3]
    elif row[0].startswith('FREIGHT'):
        if len(row) >= 2:
            freight = row[1]
    elif row[0].startswith('HANDLING'):
        if len(row) >= 2:
            invoice_handling = row[1]
    elif row[0].startswith('MISC'):
        if len(row) >= 2:
            misc = row[1]
    elif row[0].startswith('RSTK'):
        if len(row) >=3:
            restock = row[2]
    elif row[0].startswith('CODES') or row[0].startswith('TAX') or row[0].startswith('Tax'):
        if len(row) >= 6:
            tax = row[5]
    elif row[0].startswith('INVOICE') or row[0].startswith('Invoice') or row[0].startswith('CLAIM') or row[0].startswith('Claim'):
        if len(row) >= 3:
            invoice_total = row[2]
    elif len(row) >= 5 and row[0].startswith('4290') and 'CLAIM' in row[4]:
        order = row[6]
    elif len(row) >= 5 and row[0].startswith('ORDER') and 'ORDER' in row[3]:
        order = row[2]
    elif len(row) >= 2 and (row[0].startswith('PAGE:') or row[0].startswith('Page:')) and row[1].isdigit():
        page = row[1]
    elif len(row) >= 7 and row[0].startswith('Trk#:') and row[4].isdigit():
        status = row[1]
        ro = row[4]
    elif len(row) >= 7 and row[0].startswith('Trk#:') and row[5].isdigit():
        status = row[1]
        ro = row[5]
    elif i >= 16 and len(row) == 5 and row[0].startswith('Core') and row[1].startswith('Charge'):
        line = ''
        core = part_number
        part = row[0]
        number = row[1]
        status = ''
        qty_order = ''
        qty_ship = row[2]
        bo_cancel = ''
        cost = row[3]
        try:
            price = row[4]
        except IndexError:
            print(f"line 94 IndexError on row {i}: {row}")
        part_number = part + " " + number
        invoice_page = (invoice_no, page, part_number, line, core)
        if invoice_page not in invoice_pages:
            if isinstance(qty_order, str):
                if len(qty_order) > 0:
                    qty_order = float(qty_order)
                else:
                    qty_order = None
            if isinstance(qty_ship, str):
                if len(qty_ship) > 0:
                    qty_ship = float(qty_ship)
                else:
                    qty_ship = None
            if isinstance(cost, str):
                if len(cost) > 7:
                    cost = float(cost.replace(',', ''))
                elif len(cost) > 1:
                    cost = float(cost)
                else:
                    cost = None                
            if isinstance(price, str):
                if len(price) > 7:
                    price = float(price.replace(',', ''))
                elif len(price) > 1:
                    price = float(price)
                else:
                    price = None
            invoice_data = [invoice_no, date, line, order, part_number, status, qty_order, qty_ship, bo_cancel, cost, price]
            invoices.append(invoice_data)
            invoice_pages.add(invoice_page)  # add the invoice number to the set
        else:
            print(f"line 102 Duplicate invoice part row {i}: {invoice_page}")
    elif i >= 16 and len(row) == 4 and '(' in row[0] and ')' in row[2]:
        line = ''
        core = ''
        part = row[0]
        number = row[1] + ')'
        status = ''
        qty_order = row[3]
        qty_ship = ''
        bo_cancel = ''
        cost = ''
        price = ''
        part_number = part + number
        invoice_page = (invoice_no, page, part_number, line, core)
        if invoice_page not in invoice_pages:
            if isinstance(qty_order, str):
                if len(qty_order) > 0:
                    qty_order = float(qty_order)
                else:
                    qty_order = None
            if isinstance(qty_ship, str):
                if len(qty_ship) > 0:
                    qty_ship = float(qty_ship)
                else:
                    qty_ship = None
            if isinstance(cost, str):
                if len(cost) > 7:
                    cost = float(cost.replace(',', ''))
                elif len(cost) > 1:
                    cost = float(cost)
                else:
                    cost = None                
            if isinstance(price, str):
                if len(price) > 7:
                    price = float(price.replace(',', ''))
                elif len(price) > 1:
                    price = float(price)
                else:
                    price = None
            invoice_data = [invoice_no, date, line, order, part_number, status, qty_order, qty_ship, bo_cancel, cost, price]
            invoices.append(invoice_data)
            invoice_pages.add(invoice_page)  # add the invoice number to the set
            core = ''
        else:
            print(f"line 120 Duplicate invoice part row {i}: {invoice_page}")
    elif len(row) >= 3 and row[2].isdigit() and isinstance(row[2], int):
        if row[2] <= 9:
            if invoice_no.startswith('C'):
                if row[4].endswith('CR'):
                    line = ro
                    order = status
                    part = row[0]
                    number = row[1]
                    status = 'Core Return'
                    qty_order = row[2]
                    act_code = 'CR'
                    handling = ''
                    cost = row[3]
                    core = part_number
                    shipment = ''
                    try:
                        price = row[4]
                        if price.endswith('CR'):
                            price = float(price.replace(',', '')[:-2]) * -1
                        else:
                            price = float(price.replace(',', ''))
                    except IndexError:
                        print(f"line 134 IndexError on row {i}: {row}")
                invoice_page = (invoice_no, page, part_number, line, core)
                if invoice_page not in invoice_pages:
                    if isinstance(qty_order, str):
                        if len(qty_order) > 0:
                            qty_order = float(qty_order)
                        else:
                            qty_order = None
                    if isinstance(handling, str):
                        if len(handling) > 0:
                            handling = (float(handling) * -1)
                        else:
                            handling = None
                    if isinstance(cost, str):
                        if len(cost) > 7:
                            cost = (float(cost.replace(',', '')) * -1)
                        elif len(cost) > 1:
                            cost = (float(cost) * -1)
                        else:
                            cost = None                
                    if isinstance(price, str):
                        if len(price) > 7:
                            price = (float(price.replace(',', '')) * -1)
                        elif len(price) > 1:
                            price = (float(price) * -1)
                        else:
                            price = None
                    credit_data = [invoice_no, date, line, order, part_number, status, qty_order, act_code, handling, cost, price]
                    credits.append(credit_data)
                    invoice_pages.add(invoice_page)  # add the invoice number to the set
        
    elif i >= 16 and row[0].isdigit() and row[0] != '4290' and row[0] != '111':
        if len(row) <= 5:
            if invoice_no.startswith('C'):
                if row[4].endswith('CR'):
                    line = ro
                    order = status
                    part = row[0]
                    number = row[1]
                    status = 'Core Return'
                    qty_order = row[2]
                    act_code = 'CR'
                    handling = ''
                    cost = row[3]
                    core = part_number
                    shipment = ''
                    try:
                        price = row[4]
                        if price.endswith('CR'):
                            price = float(price.replace(',', '')[:-2]) * -1
                        else:
                            price = float(price.replace(',', ''))
                    except IndexError:
                        print(f"line 134 IndexError on row {i}: {row}")
        elif len(row) >= 6:
            line = row[0]
            part = row[1]
            number = row[2]
            status = row[3]
            if invoice_no.startswith('C'):
                if "CR" in row[4]:
                    line = ro
                    order = status
                    part = row[0]
                    number = row[1]
                    status = 'Core Return'
                    qty_order = row[2]
                    act_code = 'CR'
                    handling = ''
                    cost = row[3]
                    core = part_number
                    shipment = ''
                    try:
                        price = row[4]
                        if price.endswith('CR'):
                            price = float(price.replace(',', '')[:-2]) * -1
                        else:
                            price = float(price.replace(',', ''))
                    except IndexError:
                        print(f"line 155 IndexError on row {i}: {row}")
                else:
                    qty_order = row[4]
                    act_code = row[5]
                    handling = row[6]
                    cost = row[7]
                    shipment = ''
                    try:
                        price = row[8]
                    except IndexError:
                        print(f"line 164 IndexError on row {i}: {row}")
            elif invoice_no.startswith('P'):
                if not row[3].isdigit():
                    if row[3].startswith('A'):
                        qty_order = ''
                        qty_ship = row[4]
                        bo_cancel = ''
                        cost = row[5]
                        try:
                            price = row[6]
                        except IndexError:
                            print(f"line 175 IndexError on row {i}: {row}")
                    elif row[3].startswith('B'):
                        qty_order = row[4]
                        qty_ship = row[5]
                        bo_cancel = row[6]
                        cost = row[7]
                        try:
                            price = row[8]
                        except IndexError:
                            print(f"line 184 IndexError on row {i}: {row}")
                    elif row[3] == 'J':
                        qty_order = row[4]
                        qty_ship = row[5]
                        cost = row[6]
                        try:
                            price = row[7]
                        except IndexError:
                            print(f"line 192 IndexError on row {i}: {row}")
                    elif row[3] == 'H':
                        qty_order = row[4]
                        qty_ship = row[5]
                        cost = row[6]
                        try:
                            price = row[7]
                        except IndexError:
                            print(f"line 200 IndexError on row {i}: {row}")
                    elif row[3].endswith('-'):
                        row[3] = row[3][:-1]
                        row[4] = row[4][:-1]
                        row[6] = row[6][:-1]
                        status = ''
                        qty_order = (int(row[3]) * -1)
                        qty_ship = (int(row[4]) * -1)
                        bo_cancel = ''
                        cost = row[5]
                        price = '{:.2f}'.format(float(row[6].replace(',', '')) * -1)
                elif row[3].isdigit():
                    status = ''
                    qty_order = row[3]
                    qty_ship = row[4]
                    bo_cancel = ''
                    cost = row[5]
                    try:
                        price = row[6]
                    except IndexError:
                        print(f"line 210 IndexError on row {i}: {row}")
        part_number = part + number 
        if invoice_no.startswith('P'):
            invoice_page = (invoice_no, page, part_number, line, core)
            if isinstance(qty_order, str):
                if len(qty_order) > 0:
                    qty_order = float(qty_order)
                else:
                    qty_order = None
            if isinstance(qty_ship, str):
                if len(qty_ship) > 0:
                    qty_ship = float(qty_ship)
                else:
                    qty_ship = None
            if isinstance(cost, str):
                cost = float(cost.replace(',', ''))
                if isinstance(cost, float):
                    cost = float(cost)
                else:
                    cost = None                
            if isinstance(price, str):
                price = float(price.replace(',', ''))
                if isinstance(price, float):
                    price = float(price)
                else:
                    price = None
            invoice_data = [invoice_no, date, line, order, part_number, status, qty_order, qty_ship, bo_cancel, cost, price]
            invoices.append(invoice_data)
            invoice_pages.add(invoice_page)  # add the invoice number to the set
        elif invoice_no.startswith('C'):
            invoice_page = (invoice_no, page, part_number, line, core)
            if invoice_page not in invoice_pages:
                if isinstance(qty_order, str):
                    if len(qty_order) > 0:
                        qty_order = float(qty_order)
                    else:
                        qty_order = None
                if isinstance(handling, str):
                    if len(handling) > 0:
                        handling = (float(handling) * -1)
                    else:
                        handling = None
                if isinstance(cost, str):
                    if len(cost) > 7:
                        cost = (float(cost.replace(',', '')) * -1)
                    elif len(cost) > 1:
                        cost = (float(cost) * -1)
                    else:
                        cost = None                
                if isinstance(price, str):
                    if len(price) > 7:
                        price = (float(price.replace(',', '')) * -1)
                    elif len(price) > 1:
                        price = (float(price) * -1)
                    else:
                        price = None
                credit_data = [invoice_no, date, line, order, part_number, status, qty_order, act_code, handling, cost, price]
                credits.append(credit_data)
                invoice_pages.add(invoice_page)  # add the invoice number to the set
            else:
                print(f"line 229 Duplicate invoice row {i}: {invoice_page}")
    if len(prev_row) > 0 and ('INVOICE' in prev_row[0] or 'Invoice' in prev_row[0] or 'CLAIM' in prev_row[0] or 'Claim' in prev_row[0]):
        if isinstance(invoice_core, str):
            if len(invoice_core) > 7:
                invoice_core = float(invoice_core.replace(',', ''))
            elif len(invoice_core) > 0:
                invoice_core = float(invoice_core)
            else:
                invoice_core = None
        if isinstance(subtotal, str):
            if subtotal.endswith('CR'):
                subtotal = float(subtotal.replace(',', '')[:-2]) * -1
            elif subtotal.endswith('-'):
                subtotal = float(subtotal.replace(',', '')[:-1]) * -1
        if isinstance(subtotal, str):
            if len(subtotal) > 7:
                subtotal = float(subtotal.replace(',', ''))
            elif len(subtotal) > 0:
                subtotal = float(subtotal)
            else:
                subtotal = None
        if isinstance(freight, str):
            if len(freight) > 0:
                freight = float(freight)
            else:
                freight = None
        if isinstance(invoice_handling, str):
            if len(invoice_handling) > 7:
                invoice_handling = float(invoice_handling.replace(',', ''))
            elif len(invoice_handling) > 0:
                invoice_handling = float(invoice_handling)
            else:
                invoice_handling = None                
        if isinstance(misc, str):
            if len(misc) > 7:
                misc = float(misc.replace(',', ''))
            elif len(misc) > 0:
                misc = float(misc)
            else:
                misc = None
        if isinstance(restock, str):
            if len(restock) > 7:
                restock = float(restock.replace(',', ''))
            elif len(restock) > 0:
                restock = float(restock)
            else:
                restock = None
        if isinstance(tax, str):
            if len(tax) > 7:
                tax = float(tax.replace(',', ''))
            elif len(tax) > 1:
                tax = float(tax)
            else:
                tax = None
        if isinstance(invoice_total, str):
            if invoice_total.endswith('CR'):
                invoice_total = float(invoice_total.replace(',', '')[:-2]) * -1
            elif invoice_total.endswith('-'):
                invoice_total = float(invoice_total.replace(',', '')[:-1]) * -1
        if isinstance(invoice_total, str):
            if len(invoice_total) > 7:
                invoice_total = float(invoice_total.replace(',', ''))
            elif len(invoice_total) > 1:
                invoice_total = float(invoice_total)
            else:
                invoice_total = None
        if invoice_no.startswith('C'):
            if subtotal > 0:
                subtotal = subtotal * -1
            if invoice_total > 0:
                invoice_total = invoice_total * -1
            if invoice_handling != None:
                invoice_handling = invoice_handling * -1
        kia_invoices.append([invoice_no, date, shipment, invoice_core, subtotal, freight, invoice_handling, misc, restock, tax, invoice_total])
    prev_row = row

# Create a new workbook
workbook = openpyxl.Workbook()

# Create the "Invoices" sheet
invoices_sheet = workbook.active
invoices_sheet.title = 'Invoices'
invoices_sheet.append(headers)
for invoice_data in invoices:
    invoices_sheet.append(invoice_data)

# Create the "Credits" sheet
credits_sheet = workbook.create_sheet('Credits')
credits_sheet.append(credit_headers)
for credit_data in credits:
    credits_sheet.append(credit_data)

# Create the "Kia Invoices" sheet
kia_sheet = workbook.create_sheet('Kia Invoices')
kia_sheet.append(invoice_headers)
for row in kia_invoices:
    kia_sheet.append(row)

# Save the workbook
workbook.save('/content/extracted_data.xlsx')

print('Data exported successfully.')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/parts_invoices.csv already exists. Skip is set to True.
Data exported successfully.


In [31]:
import openpyxl

# Open the workbook
workbook = openpyxl.load_workbook('/content/extracted_data.xlsx')

# Get the 'Invoices' sheet
invoices_sheet = workbook['Invoices']
credits_sheet = workbook['Credits']

# Create a dictionary to store the invoice totals
invoice_totals = {}

# Iterate through the 'Invoices' sheet, adding up the price values for each invoice
for i, row in enumerate(invoices_sheet.iter_rows(min_row=1, values_only=True), start=1):
    if i == 1:
        continue  # skip header row
    invoice_no = row[0]
    price = row[10]
    if invoice_no not in invoice_totals:
        invoice_totals[invoice_no] = 0
    if price is not None:
        try:
            price = float(price)
        except ValueError:
            print(f"Could not convert price {price} to float in row {i}")
        else:
            invoice_totals[invoice_no] += price

for i, row in enumerate(credits_sheet.iter_rows(min_row=1, values_only=True), start=1):
    if i == 1:
        continue  # skip header row
    invoice_no = row[0]
    price = row[10]
    if invoice_no not in invoice_totals:
        invoice_totals[invoice_no] = 0
    if price is not None:
        try:
            price = float(price)
        except ValueError:
            print(f"Could not convert price {price} to float in row {i}")
        else:
            invoice_totals[invoice_no] += price

# Round the invoice totals to two decimal places
for invoice_no in invoice_totals:
    invoice_totals[invoice_no] = round(invoice_totals[invoice_no], 2)

# Get the 'Kia Invoices' sheet
kia_sheet = workbook['Kia Invoices']

# Iterate through the 'Kia Invoices' sheet, comparing the subtotal value to the invoice total for each invoice
for row in kia_sheet.iter_rows(min_row=1, values_only=True):
    invoice_no = row[0]
    subtotal = row[4]
    if invoice_no in invoice_totals:
        if subtotal != invoice_totals[invoice_no]:
#            print(f"Invoice {invoice_no} subtotal matches invoice total: {subtotal}")
#        else:
            print(f"Invoice {invoice_no} subtotal does not match invoice total: {subtotal} vs {invoice_totals[invoice_no]}")
    else:
        print(f"No invoice total found for invoice {invoice_no}")


No invoice total found for invoice INVOICE #
Invoice CX30354337 subtotal does not match invoice total: -22000 vs -14000.0
Invoice PL33238337 subtotal does not match invoice total: 7962.74 vs 9041.99
Invoice CX30249011 subtotal does not match invoice total: -40000 vs -32000.0
Invoice CX30246623 subtotal does not match invoice total: -39000 vs -18000.0


In [ ]:
import csv
import re
from pdfminer.high_level import extract_text

pdf_path = '/content/packing list.pdf'

# Extract text from PDF file
text = extract_text(pdf_path)

# Split text into lines and remove any empty lines
lines = [line.strip() for line in text.split('\n') if line.strip()]

# Split each line into columns using regex
columns = []
for line in lines:
    # Use regex to split line into columns
    # Here, we split the line based on tab characters and multiple spaces
    # You can adjust this regex as needed to split the line correctly
    cols = re.split(r'\t| {2,}', line)
    # Remove empty columns and append to list of columns
    columns.extend([col for col in cols if col.strip()])

# Write columns to CSV file
with open('/content/packing list.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file, delimiter='\t')
    writer.writerow(columns)


In [ ]:
import csv
import re
import datetime
from pdfminer.high_level import extract_text

pdf_path = '/content/packing list.pdf'

# Extract text from PDF file
text = extract_text(pdf_path)

# Split text into lines and remove any empty lines
lines = [line.strip() for line in text.split('\n') if line.strip()]

# Split each line into columns using regex
columns = []
for line in lines:
    # Use regex to split line into columns
    # Here, we split the line based on tab characters and multiple spaces
    # You can adjust this regex as needed to split the line correctly
    cols = re.split(r'\t| {2,}', line)
    # Remove empty columns and append to list of columns
    columns.extend([col for col in cols if col.strip()])

# Find the index of the column containing today's date
today = datetime.date.today().strftime('%m/%d/%Y')
date_col_index = None
for i, col in enumerate(columns):
    if col.endswith(today):
        date_col_index = i
        break

# If no column contains today's date, exit the script
if date_col_index is None:
    print(f"No data found for today's date: {today}")
    exit()

# Extract data from rows that end with today's date
rows = []
for line in lines:
    cols = re.split(r'\t| {2,}', line)
    # Only add the row if it ends with today's date
    if cols[date_col_index].endswith(today):
        rows.append(cols)

# Write rows to CSV file
with open('/content/packing list.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file, delimiter='\t')
    # Write header row
    writer.writerow(columns)
    # Write data rows
    for row in rows:
        writer.writerow(row)


IndexError: ignored